In [7]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient
from llama_index.core import Document, VectorStoreIndex, StorageContext, Settings
from dotenv import load_dotenv
from llama_index.embeddings.openai import OpenAIEmbedding
from qdrant_client.http.models import PayloadSchemaType
import os
from llama_index.core.vector_stores.types import VectorStoreQueryMode
load_dotenv()

client = QdrantClient(
    url=os.getenv("QDRANT_URL"), 
    api_key=os.getenv("QDRANT_API_KEY"),
)
aclient = AsyncQdrantClient(
    url=os.getenv("QDRANT_URL"), 
    api_key=os.getenv("QDRANT_API_KEY"),
) 

def config_db(collection_name):
    Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
    vector_store = QdrantVectorStore(
        collection_name=collection_name,
        client=client,
        aclient=aclient,
        enable_hybrid=True,
        fastembed_sparse_model="Qdrant/bm25",
        batch_size=20,
    )
    return vector_store

In [8]:
from llama_index.core.settings import Settings
# from llama_index.postprocessor.colbert_rerank import ColbertRerank
from llama_index.core.postprocessor import LLMRerank
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
    FilterCondition
)
import asyncio
from concurrent.futures import ThreadPoolExecutor
from llama_index.core import QueryBundle
from llama_index.core.prompts import PromptTemplate
from llama_index.core.prompts.prompt_type import PromptType
import json

In [20]:
vector_store = config_db("hello_my_friend2")
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
filename_ids = ['f_b20befa8-8035-11f0-baa1-38f3abb08dd1']
filters_chunk = MetadataFilters(
    filters=[
        MetadataFilter(key="filename_id", operator=FilterOperator.IN, value=filename_ids),
        MetadataFilter(key="type", operator=FilterOperator.EQ, value="chunk_document"),
    ],
    condition=FilterCondition.AND,
)
query_str = "Cấu hình thiết bị nguồn: Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 3"
# query_bundle = QueryBundle(query_str)
retriever_chunk = index.as_retriever(
    similarity_top_k=10,
    sparse_top_k=15,
    # verbose=True,
    vector_store_query_mode=VectorStoreQueryMode.HYBRID,
    filters=filters_chunk,
    hybrid_top_k=10
)
retrieved_nodes = retriever_chunk.retrieve(query_bundle)
reranker = LLMRerank(
    # choice_select_prompt=custom_rerank_prompt,
    choice_batch_size=10,
    top_n=5,
    llm=Settings.llm
)
results = reranker.postprocess_nodes(
    retrieved_nodes, query_bundle
)

In [21]:
content = ""
for i, result in enumerate(retrieved_nodes, start=1):
    metadata = result.metadata
    file_name = metadata["file_name"] + ".pdf"
    page = metadata["page"]
    table = metadata["table_name"]
    figure_name = metadata.get("figure_name")
    text = result.text.strip()
    content += f"""
    ========== CHUNK {metadata['chunk_id']} ========
        page: {page}
        file: {file_name}
        figure: {figure_name}
        table: {table}

        Nội dung của chunk: {text}
        ========== CHUNK {metadata['chunk_id']} END ==========\n\n"""

In [22]:
content

"\n    ========== CHUNK R48-1000e3_User_Manual_47 ========\n        page: 20\n        file: R48-1000e3_User_Manual.pdf\n        figure: Figure 6\n        table: None\n\n        Nội dung của chunk: ## Installing Rectifiers\n\nRectifiers can be inserted or removed with power applied (hot swappable).\n\nNOTE! Each rectifier module locks into a module mounting shelf by means of a latch located on the bottom of the rectifier. The latch and rectifier handle are interactive. Pushing the handle up into the rectifier's front panel causes the latch to extend to the locking position; pulling the handle down out from the rectifier's front panel causes the latch to retract. See Figure 6.\n\nCAUTION! This rectifier contains double pole fusing. Parts of the equipment that remain energized might represent a hazard during servicing after operation of the fuse. If the rectifiers are connected to a 3-phase system, the neutral line should also have a fuse.\n\nWARNING! To prevent damage to the latching mec